### Actividad: Concurrencia y ayncio

[asyncio](https://docs.python.org/3/library/asyncio.html) es una biblioteca de Python que nos permite ejecutar código utilizando un modelo de programación asíncrono. Esto nos permite manejar múltiples operaciones de E/S a la vez, al mismo tiempo que permite que nuestra aplicación siga respondiendo. 

Utiliza un modelo de concurrencia conocido como bucle de eventos de un solo subproceso (single-threaded event loop.).

#### Operaciones vinculadas a E/S y a la CPU

Cuando nos referimos a una operación como vinculada a E/S o vinculada a CPU, nos referimos al factor limitante que impide que esa operación se ejecute más rápido. Esto significa que si aumentamos el rendimiento de aquello a lo que estaba vinculada la operación, esa operación se completaría en menos tiempo. 

In [ ]:
# Operaciones vinculadas a E/S y CPU
import requests

respuestas = requests.get('https://www.example.com')

items = respuestas.headers.items()

cabeceras = [f'{key}: {header}' for key, header in items]
# Completa


### Multitarea

La multitarea está en todas partes en el mundo actual.  En esta sección se analizan dos tipos principales de multitarea: multitarea preventiva y multitarea cooperativa. 

**Multitarea preventiva**

En este modelo, dejamos que el sistema operativo decida cómo cambiar entre qué trabajo se está ejecutando actualmente mediante un proceso llamado división de tiempo (time slicing). Cuando el sistema operativo cambia de trabajo, lo llamamos apropiación. 

El funcionamiento de este mecanismo  depende del propio sistema operativo. Se logra principalmente mediante el uso de múltiples subprocesos o múltiples procesos. 

**Multitarea cooperativa**

En este modelo, en lugar de depender del sistema operativo para decidir cuándo cambiar entre qué trabajo se está ejecutando actualmente, codificamos explícitamente puntos en nuestra aplicación donde podemos permitir que se ejecuten otras tareas. Las tareas en nuestra aplicación operan en un modelo en el que cooperan, diciendo explícitamente: “Estoy pausando mi tarea por un tiempo; adelante y ejecute otras tareas”. 


asyncio utiliza la multitarea cooperativa para lograr la concurrencia.  

La multitarea cooperativa tiene ventajas sobre la multitarea preventiva.

- La multitarea cooperativa requiere menos recursos. 
- Granularidad


####  Procesos, subprocesos, multithreading y multiprocessing 

-  Un proceso es la ejecución de una aplicación que tiene un espacio de memoria al que otras aplicaciones no pueden acceder.
- Los hilos pueden considerarse procesos más livianos. Además, son la construcción más pequeña que puede gestionar un sistema operativo. No tienen memoria propia como la tiene un proceso; en cambio, comparten la memoria del proceso que los creó. Los hilos están asociados con el proceso que los creó. Un proceso siempre tendrá al menos un hilo asociado, generalmente conocido como `hilo principal`. Un proceso también puede crear otros subprocesos, que se conocen más comúnmente como subprocesos de trabajo o en segundo plano. 

In [ ]:
import os
import threading

print(f'Proceso de Python ejecutandose con un ID: {os.getpid()}')

total_threads = threading.active_count()
nombre_thread = threading.current_thread().name

print(f'Python esta corriendo {total_threads} thread(s) actualmente')
print(f'El actual thread es {nombre_thread}')

Los procesos también pueden crear otros subprocesos que comparten la memoria del proceso principal. Estos subprocesos pueden realizar otros trabajos simultáneamente a través de lo que se conoce como **multithreading**. 

In [ ]:
import threading


def hello_thread():
    print(f'Hola desde el thread {threading.current_thread()}!')


hello_thread = threading.Thread(target=hello_thread)
hello_thread.start()
# completa

El multithreaded no es la única forma en que podemos lograr la concurrencia. También podemos crear múltiples procesos para que trabajen simultáneamente para nosotros. Esto se conoce como **multiprocesamiento**. En el multiprocesamiento, un proceso padre crea uno o más procesos hijos que administra. Luego puede distribuir trabajo a los procesos hijos. 

Python nos proporciona el módulo `multiprocessing` para manejar esto. La API es similar a la del módulo de `threading`. Primero creamos un proceso con una función `target`. Luego, llamamos a su método `start` para ejecutarlo y finalmente a su método `join` para esperar a que complete su ejecución. 

In [ ]:
import multiprocessing
import os
 
 
def hello_procesos():
    print(f'Hola desde los procesos hijos {os.getpid()}!')
if __name__ == '__main__':
    hello_proceso = multiprocessing.Process(target=hello_procesos)
    hello_proceso.start()
 
    print(f'Hola desde el proceso padre {os.getpid()}')
 
    hello_proceso.join()

Multithreading y multiprocessing  pueden parecer soluciones mágicas para permitir la concurrencia con Python. Sin embargo, el poder de estos modelos de concurrencia se ve obstaculizado por un detalle de implementación de Python: el bloqueo global del intérprete (GIL).

### ¿Por que existe el GIL?

La respuesta está en cómo se gestiona la memoria en CPython. En CPython, la memoria se gestiona principalmente mediante un proceso conocido como [recuento de referencias](https://towardsdatascience.com/understanding-reference-counting-in-python-3894b71b5611) ( reference counting ). 

El recuento de referencias funciona al realizar un seguimiento de quién necesita actualmente acceso a un objeto de Python en particular, como un número entero, un diccionario o una lista. Un recuento de referencias es un número entero que registra cuántos lugares hacen referencia a ese objeto en particular. Cuando alguien ya no necesita ese objeto al que se hace referencia, el recuento de referencias disminuye y cuando alguien más lo necesita, se incrementa. Cuando el recuento de referencias llega a cero, nadie hace referencia al objeto y se puede eliminar de la memoria. 

El conflicto con los subprocesos surge porque la implementación en CPython no es segura para subprocesos. Cuando decimos que CPython no es seguro para subprocesos, queremos decir que si dos o más subprocesos modifican una variable compartida, esa variable puede terminar en un estado inesperado. Este estado inesperado depende del orden en que los subprocesos acceden a la variable, lo que comúnmente se conoce como **condición de carrera**. Las condiciones de carrera pueden surgir cuando dos subprocesos necesitan hacer referencia a un objeto Python al mismo tiempo.  

Para resolver condiciones de carrera como esta, se deben usar mecanismos de sincronización como semáforos, mutexes o bloqueos (locks).

Para demostrar el efecto del GIL en la programación multithreaded, examinemos la tarea que requiere un uso intensivo de la CPU de calcular el enésimo número de la secuencia de Fibonacci. Usaremos una implementación bastante lenta del algoritmo para demostrar una operación que requiere mucho tiempo.

In [ ]:
def print_fib(numero: int) -> None:
    def fib(n: int) -> int:
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return fib(n - 1) + fib(n - 2)

    print(f'fib({numero}) is {fib(numero)}')


def fibs_no_threading():
    print_fib(40)
    print_fib(41)


inicio = time.time()

fibs_no_threading()

final = time.time()

print(f'Completado en {final - inicio:.4f} segundos.')

Esta implementación utiliza recursividad y, en general, es un algoritmo relativamente lento que requiere un tiempo exponencial $O(2^N)$ para completarse. 

Ahora usamos multithreading en la secuencia Fibonacci:

In [ ]:
import threading
import time


def print_fib(numero: int) -> None:
    def fib(n: int) -> int:
        if n == 1:
            return 0
        elif n == 2:
            return 1
        else:
            return fib(n - 1) + fib(n - 2)

    print(f'fib({numero}) is {fib(numero)}')


def fibs_con_threads():
    thread_40 = threading.Thread(target=print_fib, args=(40,))
    thread_41= threading.Thread(target=print_fib, args=(41,))

    thread_40.start()
    thread_41.start()

    thread_40.join()
    thread_41.join()


inicio_threads = time.time()

fibs_con_threads()

fin_threads = time.time()

print(f'Threads toman {fin_threads - inicio_threads:.4f} seconds.')

Esta última versión  tomó casi la misma cantidad de tiempo. De hecho, ¡fue incluso un poco más lento! (comprueba!) Esto se debe casi en su totalidad al GIL y a la sobrecarga de crear y administrar subprocesos. Si bien es cierto que los subprocesos se ejecutan concurrentemente, solo uno de ellos puede ejecutar código Python a la vez debido al bloqueo. Esto deja al otro subproceso en estado de espera hasta que se completa el primero, lo que niega por completo el valor de varios subprocesos. 

#### ¿La concurrencia en Python puede ocurrir alguna vez con los subprocesos?

 El GIL no se mantiene para siempre, de modo que no podamos utilizar múltiples subprocesos para nuestro beneficio.  

El bloqueo global del intérprete se libera cuando ocurren operaciones de E/S. Esto nos permite emplear subprocesos para realizar trabajo concurrente en lo que respecta a E/S, pero no para el código Python vinculado a la CPU en sí (hay algunas excepciones notables que liberan el GIL para el trabajo vinculado a la CPU en ciertas circunstancias).

In [ ]:
import time
import requests


def leer_ejemplo() -> None:
    respuesta = requests.get('https://www.example.com')
    print(respuesta.status_code)

    #completa

Ahora que tenemos una base de referencia de cómo se ve una versión síncrona, podemos escribir una versión multithreaded para compararla.  

In [ ]:
import time
import threading
import requests


def leer_ejemplo() -> None:
    respuesta= requests.get('https://www.example.com')
    print(respuesta.status_code)


thread_1 = threading.Thread(target=leer_ejemplo)
thread_2 = threading.Thread(target=leer_ejemplo)

inicio_thread = time.time()

thread_1.start()
thread_2.start()

print('Todos los subprocesos corriendo!')

thread_1.join()
thread_2.join()

final_thread = time.time()

print(f'Ejecutar sincronicamente toma {final_thread - inicio_thread:.4f} segundos.')

Esto es aproximadamente dos veces más rápido que nuestra versión original que no usaba subprocesos, ya que ejecutamos las dos solicitudes aproximadamente al mismo tiempo! (comprueba).

### asyncio y el GIL

asyncio aprovecha el hecho de que las operaciones de E/S liberan el GIL para brindarnos concurrencia, incluso con un solo subproceso. Cuando utilizamos asyncio creamos objetos llamados **corutinas**.

 Es importante tener en cuenta que asyncio no elude el GIL y todavía estamos sujetos a él. Si tenemos una tarea vinculada a la CPU, aún necesitamos usar múltiples procesos para ejecutarlos concurrentemente (lo que se puede hacer con el propio asyncio); de lo contrario, causaremos problemas de rendimiento en nuestra aplicación. Ahora que sabemos que es posible lograr concurrencia para E/S con un solo subproceso, profundicemos en los detalles de cómo funciona esto con sockets sin bloqueo. 

#### ¿Qué es un socket?

Un socket es una abstracción de bajo nivel para enviar y recibir datos a través de una red. Es la base de cómo se transfieren los datos hacia y desde los servidores. Los sockets admiten dos operaciones principales: enviar bytes y recibir bytes.

Los sockets se bloquean de forma predeterminada. En pocas palabras, esto significa que cuando esperamos que un servidor responda con datos, detenemos nuestra aplicación o la bloqueamos hasta que tengamos datos para leer. Por lo tanto, nuestra aplicación deja de ejecutar cualquier otra tarea hasta que obtengamos datos del servidor, ocurra un error o se agote el tiempo de espera. 

 A nivel de sistema operativo no necesitamos hacer este bloqueo. Los sockets pueden funcionar en modo sin bloqueo. En el modo sin bloqueo, cuando escribimos bytes en un socket, podemos simplemente activar y olvidar la escritura o lectura, y nuestra aplicación puede continuar realizando otras tareas. Posteriormente, podemos hacer que el sistema operativo nos diga que recibimos bytes y lo trate en ese momento. Esto permite que la aplicación haga cualquier cantidad de cosas mientras esperamos que nos lleguen los bytes.
  

En segundo plano, esto lo realizan algunos **sistemas de notificación de eventos** diferentes, según el sistema operativo que estemos ejecutando. asyncio es lo suficientemente abstracto como para cambiar entre los diferentes sistemas de notificación, dependiendo de cuál admita nuestro sistema operativo.


Estos sistemas realizan un seguimiento de nuestros sockets sin bloqueo y nos notifican cuando están listos para que hagamos algo con ellos. Este sistema de notificación es la base de cómo asyncio puede lograr la concurrencia. En el modelo de concurrencia de asyncio, solo tenemos un hilo ejecutando Python en un momento dado. Cuando realizamos una operación de E/S, la entregamos al sistema de notificación de eventos de nuestro sistema operativo para que realice un seguimiento por nosotros. Una vez que hayamos realizado esta transferencia, nuestro hilo de Python podrá seguir ejecutando otro código de Python o agregar más sockets sin bloqueo para que el sistema operativo realice un seguimiento por nosotros. Cuando finaliza nuestra operación de E/S, "despertamos" la tarea que estaba esperando el resultado y luego procedemos a ejecutar cualquier otro código Python que vino después de esa operación de E/S. 

  

### ¿Cómo funciona un bucle de eventos?

Un bucle de eventos es la base de cada aplicación asyncio. Los bucles de eventos son un patrón de diseño bastante común en muchos sistemas y existen desde hace bastante tiempo. Si alguna vez usó JavaScript en un navegador para realizar una solicitud web asincrónica, creó una tarea en un bucle de eventos. Las aplicaciones GUI de Windows utilizan lo que se denomina bucles de mensajes detrás de escena como mecanismo principal para manejar eventos como la entrada del teclado, al tiempo que permiten que la interfaz de usuario se dibuje. 

El bucle de eventos más básico es extremadamente simple. Creamos una cola que contiene una lista de eventos o mensajes. Luego hacemos un bucle indefinido, procesando los mensajes uno a la vez a medida que entran en la cola. En Python, un bucle de eventos básico podría verse así: 

In [ ]:
####
from collections import deque
mensajes = deque()

while True:
    if mensajes:
        mensajes = mensajes.pop()
        proceso_mensajes(mensajes)

En asyncio, el bucle de eventos mantiene una cola de tareas en lugar de mensajes. Las tareas son envoltorios de una corutina. Una corutina puede pausar la ejecución cuando llega a una operación vinculada a E/S y permitirá que el bucle de eventos ejecute otras tareas que no están esperando a que se completen las operaciones de E/S. 